# Task 4.1: How to provide your services through an API?
## Creació de servidor web
Per tal de proveir servei, creem un servidor a l'escolta en el port 1993 mitjançant el qual servirem la pàgina que oferirà els serveis.

In [ ]:
import SimpleHTTPServer
import SocketServer

PORT = 1993

class MyRequestHandler(SimpleHTTPServer.SimpleHTTPRequestHandler):
    def do_GET(self): # tractem dos serveis, depenent de URL d'accés
        if self.path == '/servei_plot':
            self.path = '/OurService1.html'
        elif self.path == '/servei_geo':
            self.path = '/OurService1.html'
        return SimpleHTTPServer.SimpleHTTPRequestHandler.do_GET(self)

Handler = MyRequestHandler
server = SocketServer.TCPServer(('0.0.0.0', PORT), Handler)

print 'Started my REST API on port ' + str(PORT)
server.serve_forever()

Started my REST API on port 1993


## Tractament d'emoticones
Importem el codi de les dues pràctiques anteriors que tracten els emoticones.

In [1]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

## Càrrega de fitxer de tweets d'IA
Carreguem el fitxer de tweets d'intel·ligència artificial doncs hi ha 4 dels quals podem obtenir la geolocalització. En el nostre exemple de la Telefónica no n'hi havia cap que tingués la geolocalització activada.

In [3]:
import operator 
import json
from collections import Counter

fname = '../json/ArtificialIntelligenceTweets.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    print(count_all.most_common(5))

[(u':', 4), (u'\u2026', 4), (u'RT', 3), (u'to', 3), (u'#ArtificialIntelligence', 2)]


## Importem Vincent i creem fitxer de freqüències
Finalment mitjançant la llibreria Vicent que hem instal·lat procedim a crear el fitxer JSON amb les freqüències dels most_common per tal de passar-li com a paràmetre d'entrada de dades al nostre sistema web de printat de plot.

In [5]:
import vincent
 
word_freq = count_all.most_common(15)
labels, freq = zip(*word_freq)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('../json/term_freq.json')

# Task 4.2: How to provide our service combined with third-party services?
## Procediment
En aquest cas tractem les coordenades per a treballar la segona part de la pràctica, el printat de punts en un mapa, obtinguts mitjançant el camp de geolocalització d'un tweet. Per fer-ho usem els tweets del Racó, doncs n'hi ha 4 que tenen geolocalització mentre que en els nostres cap en té.

In [7]:
import json

fname = '../json/Lab3.CaseStudy.json'
with open(fname, 'r') as f:
    geo_data = {
        "type": "FeatureCollection",
        "features": []
    }
    for line in f:
        tweet = json.loads(line)
        if tweet['coordinates']:
            geo_json_feature = {
                "type": "Feature",
                "geometry": tweet['coordinates'],
                "properties": {
                    "text": tweet['text'],
                    "created_at": tweet['created_at']
                }
            }
            geo_data['features'].append(geo_json_feature)

with open('../json/geo_data.json', 'w') as fout:
    fout.write(json.dumps(geo_data, indent=4))

# Task 4.3: Student proposal
## Explicació
Arribats a aquest punt tenim dos serveis, el del plot i el del mapa, ambdós són accessibles mitjançant el servei web creat al servidor de la primera part de la pràctica.
## Enllaços
Servei de plot: http://localhost:1993/servei_plot

Servei de geolocalització: http://localhost:1993/servei_geo